# **Filtro para selecionar as imagens do Brasil**

In [ ]:
import os
import shutil
import rasterio
from rasterio.warp import transform_bounds
from shapely.geometry import box
import geopandas as gpd

# Carregar o shapefile dos limites do Brasil
def load_brazil_boundary():
    brazil = gpd.read_file('BR_Pais_2022/BR_Pais_2022.shp')
    brazil = brazil.to_crs(epsg=4326)  # Garantir que esteja no CRS EPSG:4326
    return brazil

# Verificar se uma imagem está dentro dos limites do Brasil
def is_image_within_brazil(image_path, brazil_boundary):
    with rasterio.open(image_path) as src:
        img_crs = src.crs
        bounds = src.bounds
        
        if img_crs.to_string() != 'EPSG:4326':
            bounds = transform_bounds(img_crs, 'EPSG:4326', *bounds)

        img_bbox = box(*bounds)
        return img_bbox.intersects(brazil_boundary)

# Função para copiar imagens que estão dentro do Brasil
def copy_images_in_brazil(src_dir, dest_dir, brazil_boundary):
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if file.endswith('.tif'):
                image_path = os.path.join(root, file)
                
                if is_image_within_brazil(image_path, brazil_boundary):
                    dest_path = os.path.join(dest_dir, file)
                    shutil.copy(image_path, dest_path)
                    print(f"Imagem {file} copiada para {dest_path}.")
                else:
                    print(f"Imagem {file} não está no Brasil.")

def main():
    src_dir = r"D:\CloudSen12Plus_\p2000\high\train"  # Diretório de origem
    dest_dir = r"D:\CloudSen12Plus_\p2000\high\train_br"  # Diretório de destino
    
    # Verificar se o diretório de destino existe, caso contrário, criar
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    brazil_boundary = load_brazil_boundary().geometry.union_all()
    copy_images_in_brazil(src_dir, dest_dir, brazil_boundary)

if __name__ == '__main__':
    main()


# **Filtro para selecionar as imagens da América do Sul**

In [ ]:
import os
import shutil
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling
from shapely.geometry import box
from rasterio.errors import RasterioIOError

# Carregar o shapefile da América do Sul
def load_south_america_shapefile(shapefile_path):
    south_america = gpd.read_file(shapefile_path)
    south_america = south_america.to_crs("EPSG:4326")  # Garantir que está em WGS84
    return south_america

# Função para verificar se a imagem cruza os limites da América do Sul
def is_in_south_america(image_path, south_america_shapefile):
    try:
        with rasterio.open(image_path) as src:
            # Reprojetar a imagem caso o CRS seja diferente do shapefile
            if src.crs != south_america_shapefile.crs:
                print(f"Reprojetando a imagem {image_path}")
                bounds = rasterio.warp.transform_bounds(src.crs, south_america_shapefile.crs, *src.bounds)
            else:
                bounds = src.bounds

            # Criar uma geometria da caixa delimitadora da imagem
            image_geom = box(bounds[0], bounds[1], bounds[2], bounds[3])

            # Verificar se há interseção com os limites da América do Sul
            intersects = south_america_shapefile.intersects(image_geom)
            return intersects.any()
    except RasterioIOError:
        print(f"Erro ao abrir a imagem: {image_path}")
        return False

# Função para criar os diretórios de destino preservando a estrutura
def create_destination_path(base_dir, relative_path):
    dest_path = os.path.join(base_dir, relative_path)
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    return dest_path

# Função principal para varrer os diretórios e copiar as imagens
def process_images(src_root, dst_root, south_america_shapefile):
    for root, dirs, files in os.walk(src_root):
        for file in files:
            if file.endswith('.tif'):
                image_path = os.path.join(root, file)
                relative_path = os.path.relpath(image_path, src_root)
                if is_in_south_america(image_path, south_america_shapefile):
                    dest_path = create_destination_path(dst_root, relative_path)
                    shutil.copy2(image_path, dest_path)
                    print(f'Imagem copiada: {image_path} -> {dest_path}')

# Caminhos das pastas de origem e destino
base_dirs = {
    r"D:\CloudSen12Plus_\p2000\train": r"D:\CloudSen12Plus_\p2000\train_as",
    r"D:\CloudSen12Plus_\p2000\val":   r"D:\CloudSen12Plus_\p2000\val_as",
    r"D:\CloudSen12Plus_\p2000\test":  r"D:\CloudSen12Plus_\p2000\test_as"
}

# Caminho para o shapefile
shapefile_path = r'Lim_america_do_sul_IBGE_2021\Lim_america_do_sul_2021.shp'

# Carregar o shapefile da América do Sul
south_america_shapefile = load_south_america_shapefile(shapefile_path)

# Processar as imagens nos diretórios train, val e test
for folder in base_dirs.keys():
    process_images(folder, base_dirs[folder], south_america_shapefile)


# **Filtro para adicionar padding nas imagens**

In [75]:
import os
import rasterio
import numpy as np

# Função para redimensionar a imagem para as dimensões-alvo
def pad_image(image, target_size):
    height, width = image.shape[1], image.shape[2]  # Considerando formato (bandas, altura, largura)
    
    if height == target_size and width == target_size:
        return image
    
    pad_height = target_size - height
    pad_width = target_size - width
    
    pad_top = 0
    pad_bottom = pad_height
    pad_left = 0
    pad_right = pad_width
    
    padded_image = np.pad(image, ((0, 0), (pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    
    return padded_image

def adjust_transform(transform, original_width, original_height, padded_width, padded_height):
    adjusted_transform = rasterio.Affine(transform.a, transform.b, transform.c, 
                                         transform.d, transform.e, 
                                         transform.f - (padded_height - original_height) * transform.e)
    return adjusted_transform

# Função para processar imagens de um diretório, incluindo subdiretórios
def process_directory(directory, target_size_small=512, target_size_large=2048):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.tif'):
                file_path = os.path.join(root, file)
                with rasterio.open(file_path) as src:
                    image = src.read()
                    
                    original_height, original_width = image.shape[1], image.shape[2]
                    
                    if original_height == 509 and original_width == 509:
                        target_size = target_size_small
                    elif original_height == 2000 and original_width == 2000:
                        target_size = target_size_large
                    else:
                        continue

                    padded_image = pad_image(image, target_size)
                    
                    new_transform = adjust_transform(src.transform, original_width, original_height, target_size, target_size)
                    
                    profile = src.profile
                    profile.update({
                        'height': target_size,
                        'width': target_size,
                        'transform': new_transform
                    })
                    
                   
                with rasterio.open(file_path, 'w', **profile) as dst:
                    dst.write(padded_image)
                    print(f"Imagem {file_path} redimensionada para {target_size}x{target_size}.")

# Caminho base onde o processo irá começar
base_dir = r"D:\CloudSen12_America_do_Sul"

# Chama a função para processar o diretório base e todos os subdiretórios
process_directory(base_dir)

print("Redimensionamento concluído!")


Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20190223T152641_20190223T152820_T18NWL__ROI_0836.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20190223T152641_20190223T152820_T18NWL__ROI_0837.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20190329T150721_20190329T150905_T19PFL__ROI_4357.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20190708T130251_20190708T130252_T24MUA__ROI_0070.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20191216T132229_20191216T132227_T23MMP__ROI_4360.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20200120T140639_20200120T141746_T19FDA__ROI_0736.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20200313T133221_20200313T133220_T22JDQ__ROI_3447.tif redimensionada para 2048x2048.
Imagem D:\CloudSen12_America_do_Sul\p2000\test_as\20200508T130

# **Filtro para quebrar as imagens de 2000 em 512**

In [1]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window

# Função para adicionar padding
def pad_image(image, target_size=(512, 512)):
    padded_image = np.zeros((image.shape[0], target_size[0], target_size[1]), dtype=image.dtype)
    padded_image[:, :image.shape[1], :image.shape[2]] = image
    return padded_image

# Função para dividir e salvar as imagens
def split_and_save(image_path, output_dir):
    # Abrir a imagem com rasterio
    with rasterio.open(image_path) as src:
        bands, height, width = src.count, src.height, src.width
        assert height == 2000 and width == 2000, "As imagens devem ter dimensão 2000x2000."

        # Definir as janelas (partes 1, 2, 3, 4)
        windows = [
            (0, 0, 500, 500),        # Parte 1
            (0, 500, 500, 1000),     # Parte 2
            (500, 0, 1000, 500),     # Parte 3
            (500, 500, 1000, 1000)   # Parte 4
        ]

        # Ler e salvar cada parte com padding
        for i, (row_start, col_start, row_end, col_end) in enumerate(windows, 1):
            window = Window(col_start, row_start, col_end - col_start, row_end - row_start)
            part = src.read(window=window)

            # Adicionar padding
            padded_part = pad_image(part)

            # Criar o nome de arquivo com sufixo
            output_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_parte_{i}.tif")

            # Escrever a imagem com padding
            with rasterio.open(
                output_path, 'w',
                driver='GTiff',
                height=padded_part.shape[1],
                width=padded_part.shape[2],
                count=bands,
                dtype=padded_part.dtype,
                crs=src.crs,
                transform=src.window_transform(window)
            ) as dst:
                dst.write(padded_part)

# Função para processar todos os arquivos dentro dos diretórios de treino, validação e teste
def process_directories(input_base_dir, output_base_dir, subdirs=['train_br', 'val_br', 'test_br']):
    for subdir in subdirs:
        input_dir = os.path.join(input_base_dir, subdir)
        output_dir = os.path.join(output_base_dir, subdir)

        # Criar o diretório de saída se não existir
        os.makedirs(output_dir, exist_ok=True)

        # Percorrer todos os arquivos .tif no diretório
        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.endswith('.tif'):
                    image_path = os.path.join(root, file)
                    # Criar o diretório correspondente para salvar as imagens quebradas
                    output_image_dir = os.path.join(output_dir, os.path.relpath(root, input_dir))
                    os.makedirs(output_image_dir, exist_ok=True)
                    
                    # Dividir e salvar as imagens
                    split_and_save(image_path, output_image_dir)

# Exemplo de uso
input_base_dir = r'D:\CloudSen12_Br - Original\p2000'  # Contém 'train', 'val', 'test'
output_base_dir = r'C:\Users\alanb\Documents\CloudSen12_Br_Resized\p2000'  # Onde serão salvas as imagens quebradas
process_directories(input_base_dir, output_base_dir)


# **Filtro para organizar as imagens baseado no percentual de nuvem**

In [ ]:
import os
import shutil
import numpy as np
import rasterio

# Função para calcular o percentual de nuvens baseado nos rótulos 1 e 2 (nuvem e nuvem fina)
def calculate_cloud_percentage(mask_band):
    # Contar quantos pixels estão nos rótulos de nuvem (1 e 2)
    cloud_pixels = np.sum(np.isin(mask_band, [1, 2]))
    total_pixels = mask_band.size

    # Calcular o percentual de nuvem
    cloud_percentage = (cloud_pixels / total_pixels) * 100
    return cloud_percentage

# Função para mover as imagens para os diretórios apropriados com base no percentual de nuvens
def move_image_based_on_cloud_percentage(image_path, mask_band, output_base_dir):
    cloud_percentage = calculate_cloud_percentage(mask_band)

    # Definir o diretório de destino com base no percentual de nuvens
    if cloud_percentage == 0:
        folder = "limpo"
    elif 0 < cloud_percentage <= 25:
        folder = "quase_claro"
    elif 25 < cloud_percentage <= 45:
        folder = "pouco_nublado"
    elif 45 < cloud_percentage <= 65:
        folder = "meio_nublado"
    else:
        folder = "nublado"

    # Criar o diretório de destino se não existir
    destination_dir = os.path.join(output_base_dir, folder)
    os.makedirs(destination_dir, exist_ok=True)

    # Mover a imagem para o diretório correto
    destination_path = os.path.join(destination_dir, os.path.basename(image_path))
    shutil.move(image_path, destination_path)
    print(f"Imagem movida para {destination_path} (percentual de nuvens: {cloud_percentage:.2f}%)")

# Função para processar todas as imagens .tif nos diretórios de train, val e test
def process_images(input_base_dir, output_base_dir, subdirs=['train_br', 'val_br', 'test_br'], cloud_mask_band_index=14):
    for subdir in subdirs:
        input_dir = os.path.join(input_base_dir, subdir)

        # Percorrer todos os arquivos .tif no diretório
        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.endswith('.tif'):
                    image_path = os.path.join(root, file)

                    # Abrir a imagem e ler a banda da máscara de nuvem
                    with rasterio.open(image_path) as src:
                        # Ler a banda da máscara de nuvem (índice fornecido)
                        mask_band = src.read(cloud_mask_band_index)  # Banda da máscara de nuvem

                    # Mover a imagem para o diretório correspondente com base no percentual de nuvens
                    move_image_based_on_cloud_percentage(image_path, mask_band, output_base_dir)

# Exemplo de uso
input_base_dir = r'C:\Users\alanb\Documents\CloudSen12_Br_Resized\p2000'  # Diretório contendo as pastas 'train', 'val' e 'test'
output_base_dir = r'C:\Users\alanb\Documents\CloudSen12_Br_Resized\p2000'  # Diretório onde as imagens classificadas serão movidas

process_images(input_base_dir, output_base_dir, cloud_mask_band_index=14)


## **Dividir as imagens em train, val e test** 

In [ ]:
import os
import shutil
import random
from math import floor

# Função para dividir as imagens em train, val e test
def split_images_into_train_val_test(base_dir, categories, output_base_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    for category in categories:
        category_dir = os.path.join(base_dir, category)

        # Verificar se o diretório da categoria existe
        if not os.path.exists(category_dir):
            print(f"Diretório {category_dir} não encontrado. Pulando...")
            continue

        # Listar todas as imagens no diretório da categoria
        all_images = [f for f in os.listdir(category_dir) if f.endswith('.tif')]

        # Embaralhar aleatoriamente as imagens
        random.shuffle(all_images)

        # Calcular quantas imagens vão para cada subconjunto
        total_images = len(all_images)
        train_count = floor(total_images * train_ratio)
        val_count = floor(total_images * val_ratio)
        test_count = total_images - train_count - val_count  # Restante vai para test

        # Dividir as imagens
        train_images = all_images[:train_count]
        val_images = all_images[train_count:train_count + val_count]
        test_images = all_images[train_count + val_count:]

        # Função auxiliar para mover as imagens para o diretório de destino
        def move_images(image_list, destination_subdir):
            destination_dir = os.path.join(output_base_dir, destination_subdir, category)
            os.makedirs(destination_dir, exist_ok=True)

            for image in image_list:
                src_image_path = os.path.join(category_dir, image)
                dest_image_path = os.path.join(destination_dir, image)
                shutil.move(src_image_path, dest_image_path)
                print(f"Imagem {image} movida para {dest_image_path}")

        # Mover as imagens para os diretórios 'train', 'val' e 'test'
        move_images(train_images, 'train_br')
        move_images(val_images, 'val_br')
        move_images(test_images, 'test_br')

# Exemplo de uso
base_dir = r'C:\Users\alanb\Documents\CloudSen12_Br_Resized\p2000'  # Diretório contendo os diretórios 'limpo', 'quase_claro', etc.
output_base_dir = r'C:\Users\alanb\Documents\CloudSen12_Br_Resized\p2000'  # Diretório onde as imagens divididas serão movidas

categories = ['limpo', 'quase_claro', 'pouco_nublado', 'meio_nublado', 'nublado']

split_images_into_train_val_test(base_dir, categories, output_base_dir)


In [82]:
import glob
import pandas as pd
train_files_512 = glob.glob(r"D:\CloudSen12_America_do_Sul\p509\nolabel\train_as\*\*\*.tif")
val_files_512 = glob.glob(r"D:\CloudSen12_America_do_Sul\p509\nolabel\val_as\*\*\*.tif")
train_df = pd.DataFrame(train_files_512, columns=['file_path'])
val_df = pd.DataFrame(val_files_512, columns=['file_path'])
print(f"Train Shape {train_df.shape}")
print(f"Val Shape {val_df.shape}")

Train Shape (2310, 1)
Val Shape (705, 1)
